In [11]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
import statistics
import cartopy.crs as crs
import cartopy.feature as cfeature
import xarray as xr
import glob

In [22]:
def format_df(df):
    """Format a DataFrame with counts of values into a DataFrame with individual values

    Args:
    - df: A pandas DataFrame with columns "COUNT" and "VALUE"

    Returns:
    - A new pandas DataFrame with a single column "VALUE" containing individual values

    Example:
    input DataFrame:

    | COUNT | VALUE |
    |-------|-------|
    |   2   |  10   |
    |   3   |  20   |
    |   1   |  30   |

    output DataFrame:

    | VALUE |
    |-------|
    |  10   |
    |  10   |
    |  20   |
    |  20   |
    |  20   |
    |  30   |
    """

    new_df = pd.DataFrame()
    value_list = []

    # iterate over the rows of the input DataFrame
    for x, _ in df.iterrows():
        # extract the count and value from each row
        count = int(df.iloc[x]["COUNT"])
        value = df.iloc[x]["VALUE"]

        # repeat the value the specified number of times
        for n in np.arange(count):
            val = value
            value_list.append(val)

    # create a new DataFrame with the individual values
    new_df["VALUE"] = value_list

    return new_df


def stat_anal(directory, state_df, station_list, lonlist, latlist):
    """
    Performs statistical analysis on a set of elevation data for a given state.

    Args:
    directory (str): File directory where the elevation data is stored.
    state_df (pandas DataFrame): DataFrame containing the elevation data for each station in the state.
    station_list (list): List of station IDs.
    lonlist (list): List of longitude coordinates for each station.
    latlist (list): List of latitude coordinates for each station.

    Returns:
    final_df (pandas DataFrame): DataFrame containing the results of the statistical analysis.
    Columns include: "station", "elev", "std", "variance", "skew", "med_dist", "lon", and "lat".
    """

    final_df = pd.DataFrame()
    std_list = []
    variance_list = []
    skew_list = []
    distance_list = []
    stations = []
    elevs = []
    x = 0
    for i in np.arange(1, 127):
        # read in csv
        df2 = pd.DataFrame()
        elev_df = pd.read_csv(f"{directory}/gfs/{i}_csv.csv")
        dfv1 = format_df(elev_df)  # apply format_df to the elevation data
        std = statistics.stdev(dfv1["VALUE"])  # calculate the standard deviation
        variance = statistics.pvariance(dfv1["VALUE"])  # calculate the variance
        my_skew = skew(dfv1["VALUE"])  # calculate the skewness
        elevation = state_df["elev"].iloc[
            x
        ]  # get the elevation for the current station
        station = station_list[x]  # get the station ID for the current station
        split_diff = (
            dfv1["VALUE"] - state_df["elev"].iloc[x]
        )  # calculate the difference between elevation and state_df
        diff_list = split_diff.to_list()  # convert the difference to a list
        df2["diff_elev"] = diff_list  # add the difference to the DataFrame
        describe = df2[
            "diff_elev"
        ].describe()  # calculate the descriptive statistics for the difference
        fifty = describe[5]  # get the median of the difference
        distance = state_df["elev"].iloc[x] - fifty  # calculate the median distance
        # add data to lists
        stations.append(station)
        elevs.append(elevation)
        distance_list.append(distance)
        skew_list.append(my_skew)
        variance_list.append(variance)
        std_list.append(std)
        x += 1

    final_df["station"] = stations
    final_df["elev"] = elevs
    final_df["std"] = std_list
    final_df["variance"] = variance_list
    final_df["skew"] = skew_list
    final_df["med_dist"] = distance_list
    final_df["lon"] = lonlist
    final_df["lat"] = latlist
    return final_df

In [14]:
ny_df = pd.read_csv("/home/aevans/nwp_bias/src/landtype/data/nysm.csv")
station_list_ny = ny_df["stid"].to_list()

In [25]:
# Initialize an empty DataFrame to store the combined data.
df_complete = pd.DataFrame()

# Loop over the CSV files and combine them into a single DataFrame.
for i in range(1, len(station_list_ny) + 1):
    # Read the CSV file and calculate the percentage of each aspect.
    elev_df = pd.read_csv(
        f"/home/aevans/nwp_bias/src/landtype/elevation/data/CSVs_elevation_ny_nam/aspect_csv_{i}.csv"
    )
    # format raw elev df
    elev_df = format_df(elev_df)

    # bucket elev df
    elev_df["rounded"] = round(elev_df["VALUE"] / 20) * 20

    the_df = pd.DataFrame()
    buckets = elev_df["rounded"].unique().tolist()
    print(buckets)
    the_df["VALUE"] = buckets

    counts_ls = []
    for value in buckets:
        counts = elev_df["rounded"].value_counts()[value]
        counts_ls.append(counts)

    the_df["COUNT"] = counts_ls

    # get percentage of bucket
    the_df = the_df.assign(
        Percentage=lambda x: (x["COUNT"] / sum(the_df["COUNT"]) * 100)
    )

    # Add the site number and pivot the DataFrame to have aspects as columns.
    the_df["site"] = i
    the_df = the_df.pivot(index="site", columns="VALUE", values="Percentage")

    # Concatenate the current DataFrame with the combined DataFrame.
    df_complete = pd.concat([df_complete, the_df])

# Add the station names as a column in the DataFrame and fill missing values with 0.
df_complete["station"] = station_list_ny
df_complete = df_complete.fillna(0)
df_complete

[340.0, 360.0, 380.0, 400.0, 420.0, 440.0, 460.0, 480.0, 500.0, 520.0, 540.0]
[400.0, 440.0, 480.0, 500.0, 520.0, 540.0, 560.0, 580.0, 600.0, 620.0, 640.0, 660.0, 680.0, 700.0, 720.0, 740.0, 760.0, 780.0, 800.0]
[200.0, 220.0, 240.0, 260.0, 280.0, 300.0]
[0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0, 140.0, 160.0, 180.0, 200.0, 240.0, 280.0, 300.0, 340.0, 360.0, 400.0, 440.0]
[300.0, 320.0, 340.0, 360.0, 380.0, 400.0, 420.0, 440.0, 460.0, 480.0, 500.0, 520.0]
[100.0, 120.0, 140.0, 160.0, 180.0]
[400.0, 420.0, 440.0, 460.0, 480.0, 500.0, 520.0, 540.0, 560.0, 580.0, 600.0, 620.0, 640.0]
[340.0, 360.0, 380.0, 400.0, 420.0, 440.0, 460.0, 480.0, 520.0]
[280.0, 300.0, 320.0, 340.0, 360.0, 380.0, 400.0, 420.0, 440.0, 460.0, 480.0, 500.0, 520.0, 540.0]
[0.0, 20.0, 40.0]
[180.0, 200.0, 220.0, 240.0, 260.0, 280.0]
[120.0, 140.0, 160.0, 180.0, 200.0, 220.0, 240.0, 260.0, 280.0]
[120.0, 140.0, 160.0, 180.0, 200.0]
[0.0, 20.0, 40.0, 60.0, 80.0, 100.0, 120.0]
[360.0, 380.0, 400.0, 420.0, 440.0, 460.0, 

VALUE,340.0,360.0,380.0,400.0,420.0,440.0,460.0,480.0,500.0,520.0,...,960.0,1000.0,1080.0,1060.0,980.0,1100.0,1120.0,1160.0,1220.0,station
site,,,,,,,,,,,,,,,,,,,,,
1,3.539823,1.769912,1.769912,5.309735,4.424779,15.929204,12.389381,21.238938,22.123894,9.734513,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,ADDI
2,0.000000,0.000000,0.000000,0.892857,0.000000,1.785714,0.000000,2.678571,1.785714,5.357143,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,ANDE
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,BATA
4,0.892857,0.892857,0.000000,0.892857,0.000000,0.892857,0.000000,0.000000,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,BEAC
5,1.769912,5.309735,7.079646,8.849558,8.849558,24.778761,15.044248,15.929204,7.079646,3.539823,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,BELD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,2.678571,3.571429,0.000000,8.928571,0.892857,2.678571,0.000000,2.678571,3.571429,0.892857,...,0.0,1.785714,0.0,0.0,2.678571,0.892857,2.678571,1.785714,0.892857,WFMB
123,0.000000,4.347826,3.478261,15.652174,27.826087,17.391304,15.652174,9.565217,4.347826,1.739130,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,WGAT
124,0.900901,4.504505,1.801802,2.702703,0.000000,0.000000,0.000000,0.900901,0.000000,0.000000,...,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,WHIT


In [16]:
df_complete.to_csv(
    "/home/aevans/nwp_bias/src/landtype/data/nysm_representativeness_analysis/clean/all_sites_csv.csv"
)

In [17]:
directory = os.listdir(
    "/home/aevans/nwp_bias/src/landtype/data/nysm_representativeness_analysis/elev"
)

In [20]:
for file in directory:
    # import raw elevation dataframe
    elev_df = pd.read_csv(
        f"/home/aevans/nwp_bias/src/landtype/data/nysm_representativeness_analysis/elev/{file}"
    )

    # format raw elev df
    elev_df = format_df(elev_df)

    # bucket elev df
    elev_df["rounded"] = round(elev_df["Value"] / 20) * 20

    the_df = pd.DataFrame()
    buckets = elev_df["rounded"].unique().tolist()
    the_df["Value"] = buckets

    counts_ls = []
    for value in buckets:
        counts = elev_df["rounded"].value_counts()[value]
        counts_ls.append(counts)

    the_df["Count"] = counts_ls
    the_df

    # get percentage of bucket
    the_df = the_df.assign(
        Percentage=lambda x: (x["Count"] / sum(the_df["Count"]) * 100)
    )
    the_df
    # format and save for climdiv

    the_df.to_csv(
        f"/home/aevans/nwp_bias/src/landtype/data/nysm_representativeness_analysis/clean/elev/{file}"
    )

In [4]:
def current_time_mesonet_df(mesonet_data_path) -> pd.DataFrame:
    """
    This will return a dataframe that contains data from the mesonet sites

    Args:
        Mesonet Data Path (f string)

    Returns:
        df (pd.DataFrame): Mesonet Data Frame
    """

    # most recent year
    dir_Year = os.listdir(f"{mesonet_data_path}")
    sort_dir_Year = sorted(dir_Year)
    data_point_Year = sort_dir_Year[-1]

    # find most recent month
    dir_Month = os.listdir(f"{mesonet_data_path}/{data_point_Year}")
    sort_dir_Month = sorted(dir_Month)
    data_point_Month = sort_dir_Month[-1]

    # this is your directory for most recent year and month
    most_recent = os.listdir(
        f"{mesonet_data_path}/{data_point_Year}/{data_point_Month}"
    )

    # most recent datapoint
    sort_most_recent = sorted(most_recent)
    data_point = sort_most_recent[-1]

    # this will return the year of the most recent data point
    new_year = data_point[0:4]

    # this will return the month of the most recent datapoint
    new_month = data_point[4:6]

    # this will return the day of the most recent datapoint
    new_day = data_point[6:8]

    # create Mesonet DataFrame

    # year
    year = new_year

    # month
    month = new_month

    # day
    day = new_day

    # file path
    file = year + month + day + ".nc"

    mesonet_df = (
        xr.open_dataset(f"{mesonet_data_path}/{year}/{month}/{file}")
        .to_dataframe()
        .reset_index()
    )
    return mesonet_df

In [5]:
def most_recent_time(df: pd.DataFrame, mesonet_data_path) -> pd.DataFrame:
    """
    This will return a dataframe that contains only the timestamps with filled data from the mesonet sites

    Args:
    Mesonet Data Path (f string)

    Returns:
    df (pd.DataFrame): Mesonet Data Frame
    """

    # most recent year
    dir_Year = os.listdir(f"{mesonet_data_path}")
    sort_dir_Year = sorted(dir_Year)
    data_point_Year = sort_dir_Year[-1]

    # find most recent month
    dir_Month = os.listdir(f"{mesonet_data_path}/{data_point_Year}")
    sort_dir_Month = sorted(dir_Month)
    data_point_Month = sort_dir_Month[-1]

    # this is your directory for most recent year and month
    most_recent = os.listdir(
        f"{mesonet_data_path}/{data_point_Year}/{data_point_Month}"
    )

    # most recent datapoint
    sort_most_recent = sorted(most_recent)
    data_point = sort_most_recent[-1]

    # this will return the year of the most recent data point
    new_year = data_point[0:4]

    # this will return the month of the most recent datapoint
    new_month = data_point[4:6]

    # this will return the day of the most recent datapoint
    new_day = data_point[6:8]

    # create Mesonet DataFrame

    # year
    year = new_year

    # month
    month = new_month

    # day
    day = new_day

    current_time_df = df.dropna(subset=["tair"])

    last_value = current_time_df["time_5M"].iat[-1]
    hour = last_value.hour
    minute = last_value.minute
    second = last_value.second

    string_hour = str(hour)
    string_minute = str(minute)
    string_sec = str(second)

    # time
    time = string_hour + ":" + string_minute + ":" + string_sec
    df.reset_index(inplace=True)

    # creating a new dataframe that is centered on the location in the dataframe
    mesonet_single_datetime_df = df.loc[df["time_5M"] == f"{year}-{month}-{day} {time}"]
    return mesonet_single_datetime_df

In [6]:
# This will return the most recent data avail on mesonet
# this is my file path
ny_df = pd.read_csv("/home/aevans/nwp_bias/src/landtype/notebooks/nysm_coords.csv")

In [7]:
# This will return the most recent data avail on mesonet
# this is my file path
ny_mesonet_data_path = "/home/aevans/nysm/archive/nysm/netcdf/proc"

In [8]:
nysm_df1 = current_time_mesonet_df(ny_mesonet_data_path)
nysm_df = most_recent_time(nysm_df1, ny_mesonet_data_path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/aevans/nysm/archive/nysm/netcdf/proc'

In [ ]:
nysm_df

In [ ]:
ny_df["elev"] = nysm_df["elev"].to_list()
ny_df

In [13]:
directory = os.listdir(f"/home/aevans/nwp_bias/src/landtype/elevation/data/NY/elev/nam")
sorted_direct = sorted(directory)

In [14]:
sorted_direct

['ADDI_elev.csv',
 'ANDE_elev.csv',
 'BATA_elev.csv',
 'BEAC_elev.csv',
 'BELD_elev.csv',
 'BELL_elev.csv',
 'BELM_elev.csv',
 'BERK_elev.csv',
 'BING_elev.csv',
 'BKLN_elev.csv',
 'BRAN_elev.csv',
 'BREW_elev.csv',
 'BROC_elev.csv',
 'BRON_elev.csv',
 'BROO_elev.csv',
 'BSPA_elev.csv',
 'BUFF_elev.csv',
 'BURD_elev.csv',
 'BURT_elev.csv',
 'CAMD_elev.csv',
 'CAPE_elev.csv',
 'CHAZ_elev.csv',
 'CHES_elev.csv',
 'CINC_elev.csv',
 'CLAR_elev.csv',
 'CLIF_elev.csv',
 'CLYM_elev.csv',
 'COBL_elev.csv',
 'COHO_elev.csv',
 'COLD_elev.csv',
 'COPA_elev.csv',
 'COPE_elev.csv',
 'CROG_elev.csv',
 'CSQR_elev.csv',
 'DELE_elev.csv',
 'DEPO_elev.csv',
 'DOVE_elev.csv',
 'DUAN_elev.csv',
 'EAUR_elev.csv',
 'EDIN_elev.csv',
 'EDWA_elev.csv',
 'ELDR_elev.csv',
 'ELLE_elev.csv',
 'ELMI_elev.csv',
 'ESSX_elev.csv',
 'FAYE_elev.csv',
 'FRED_elev.csv',
 'GABR_elev.csv',
 'GFAL_elev.csv',
 'GFLD_elev.csv',
 'GROT_elev.csv',
 'GROV_elev.csv',
 'HAMM_elev.csv',
 'HARP_elev.csv',
 'HARR_elev.csv',
 'HART_ele

In [15]:
# paths to data
path_ny = f"/home/aevans/nwp_bias/src/landtype/elevation/data/CSVs_elevation_ny_30km"

In [16]:
station_list_ny = ny_df["station"].to_list()
ny_df_lons = ny_df["longitude"].to_list()
ny_df_lats = ny_df["latitude"].to_list()

In [17]:
# x = 0
# for i in range(1, 127):
#     df = pd.read_csv(
#         f"/home/aevans/nwp_bias/src/landtype/elevation/data/CSVs_slope_ny_gfs/aspect_csv_{i}.csv"
#     )
#     df.to_csv(
#         f"/home/aevans/nwp_bias/src/landtype/elevation/data/NY/elev/gfs/{station_list_ny[x]}_elev.csv"
#     )
#     x += 1

In [18]:
slope_df = stat_anal(sorted_direct, ny_df, station_list_ny, ny_df_lons, ny_df_lats)

OSError: [Errno 36] File name too long: "['ADDI_elev.csv', 'ANDE_elev.csv', 'BATA_elev.csv', 'BEAC_elev.csv', 'BELD_elev.csv', 'BELL_elev.csv', 'BELM_elev.csv', 'BERK_elev.csv', 'BING_elev.csv', 'BKLN_elev.csv', 'BRAN_elev.csv', 'BREW_elev.csv', 'BROC_elev.csv', 'BRON_elev.csv', 'BROO_elev.csv', 'BSPA_elev.csv', 'BUFF_elev.csv', 'BURD_elev.csv', 'BURT_elev.csv', 'CAMD_elev.csv', 'CAPE_elev.csv', 'CHAZ_elev.csv', 'CHES_elev.csv', 'CINC_elev.csv', 'CLAR_elev.csv', 'CLIF_elev.csv', 'CLYM_elev.csv', 'COBL_elev.csv', 'COHO_elev.csv', 'COLD_elev.csv', 'COPA_elev.csv', 'COPE_elev.csv', 'CROG_elev.csv', 'CSQR_elev.csv', 'DELE_elev.csv', 'DEPO_elev.csv', 'DOVE_elev.csv', 'DUAN_elev.csv', 'EAUR_elev.csv', 'EDIN_elev.csv', 'EDWA_elev.csv', 'ELDR_elev.csv', 'ELLE_elev.csv', 'ELMI_elev.csv', 'ESSX_elev.csv', 'FAYE_elev.csv', 'FRED_elev.csv', 'GABR_elev.csv', 'GFAL_elev.csv', 'GFLD_elev.csv', 'GROT_elev.csv', 'GROV_elev.csv', 'HAMM_elev.csv', 'HARP_elev.csv', 'HARR_elev.csv', 'HART_elev.csv', 'HERK_

In [ ]:
slope_df

In [ ]:
slope_df.to_csv("/home/aevans/nwp_bias/src/correlation/data/elev_gfs.csv")